### UC Berkeley, MICS, W202-Cryptography
### Week 01 Breakout 4

### RSA Asymmetric Cryptography

RSA is based on finding two large prime numbers p and q and multiplying them to calculate n, essentially n = pq. n needs to be large enough such that it is computationally intractible to factor n into p and q.

We have two keys, one for encryption, denoted e, and one for decryption denoted d.  

e must be odd and realtively prime to Euler's phi = (p-1)(q-1).  Note that e does NOT have to be prime (a very common misconception)!  We typically use a large odd number randomly generated for e, although there are special exceptions where we want to choose e.

Using the Extended Euclidean Algorithm, we find a raw d as the product of ed in Bezout's Identity.  We must then find d = raw d (mod phi)

To encrypt, where m is our message in plaintext and c is our encrypted cypertext, c = m^e (mod n)

To decrypt, m = c^d (mod n)

It's two main purposes are secure key establishment and digital certificates.

For secure key establishment, the message must be an integer, and there will be a finite limit.  It's typically used to establish a key that will be used for a symmetric cryptography.  It's typical to pass a 256 bit key to be used for AES-256 symmetric cryptography. n, e, c are publically known.  p, q, d, m are secret.

For digital certifcates, the message must also be an integer.  To create a certificate we encrypt a message with e.  To verify a certificate, we decrypt it with d.  n, d, c, m are publicly known.  p, q, e are secret.

Sizing of e and d.  Typically we want to have e and d about the same size, so both encrytion and decryption take about the same amount of time. When e is small, d will be large, and vice versa.  Sometimes there are valid reasons for making either e or d small, we call these "short exponents" as opposed to "long exponets".  Short exponents (3, 7, 63537) should ONLY be used with public keys!  Since short exponents should only be used with public keys, they are usually called "short public exponents"  If short exponents are used with private keys, RSA can easily be broken!



In [1]:
from sage.all import *

In [2]:
def my_print_number(label, x):
    "prints a number in decimal, number of digits, hex, number of bits"
    
    print ("\n", label, '\n')
    print ("decimal:", "{:,}".format(x), "\n")
    print ("number of digits:", x.ndigits(), "\n")
    print ("hex:", x.hex(), "\n")
    print ("number of bits:", x.nbits(), "\n")

In [3]:
def my_find_n_p_q(b):
    "find two primes p and q that when multiplied together yield n of the given number of bits"
    
    b_half = b // 2
    
    upper_limit = (2^b_half) - 1
    lower_limit = (2^(b_half-1))
    
    p = random_prime(upper_limit, false, lower_limit)
    q = random_prime(upper_limit, false, lower_limit)
    
    n = p * q
    
    my_print_number("p", p)
    my_print_number("q", q)
    my_print_number("n", n)
    
    return (p, q, n)

In [4]:
# come back here and re-run with the remaining bit sizes

(p, q, n) = my_find_n_p_q(64)

#(p, q, n) = my_find_n_p_q(128)
#(p, q, n) = my_find_n_p_q(256)
#(p, q, n) = my_find_n_p_q(512)
#(p, q, n) = my_find_n_p_q(1024)
#(p, q, n) = my_find_n_p_q(2048)
#(p, q, n) = my_find_n_p_q(4096)



 p 

decimal: 2,901,354,377 

number of digits: 10 

hex: acef2789 

number of bits: 32 


 q 

decimal: 2,296,471,421 

number of digits: 10 

hex: 88e15f7d 

number of bits: 32 


 n 

decimal: 6,662,877,408,973,759,717 

number of digits: 19 

hex: 5c774bb3b7db24e5 

number of bits: 63 



In [5]:
phi = (p - 1) * (q - 1)

In [6]:
my_print_number("phi", phi)


 phi 

decimal: 6,662,877,403,775,933,920 

number of digits: 19 

hex: 5c774bb2820a9de0 

number of bits: 63 



In [7]:
def my_find_e(phi):
    "given Euler's phi, find an e as a random odd number relatively prime to phi"
    
    upper_limit = phi * 0.9
    lower_limit = phi * 0.5
    
    e_found = False
    
    while not e_found:
        
        e = Integer(randint(lower_limit, upper_limit))
        
        if is_even(e):
            e += 1
        
        if gcd(e,phi) == 1:
            e_found = True
    
    return e

In [8]:
def my_verify_e(e, phi):
    "in some cases, we want to give an e instead of finding one.  this will verify an e is relatively prime to phi"
    
    if gcd(e, phi) == 1:
        print ("e is valid")
    else:
        print ("e is NOT valid, do NOT use!")

In [9]:
e = my_find_e(phi)

In [10]:
# sometimes we may want to set an e as discussed previously.  
# this is often used for the so called "short exponents" (3, 7, 65537) as opposed to "long exponents"
# since short exponents should only be used with public keys, they are often called "short public exponents"

# WARNING: short exponents can only be used with public keys.  
# If you use a short exponent with a private key, RSA can easily be broken

# e = Integer(3)
# e = Integer(7)
# e = Integer(65537)

# if you use your own e, you should always validate it

# my_verify_e(e, phi)

In [11]:
my_print_number("e", e)


 e 

decimal: 5,066,291,434,781,220,101 

number of digits: 19 

hex: 464f1537b2bd1d05 

number of bits: 63 



In [12]:
def my_calculate_d(e,phi):
    "given e and phi calculate d"
    
    (g, d, b) = xgcd(e,phi)
    
    print ("\nd * e + b * phi = gcd(e,phi) = 1")
    print ("\n(" + str(d) + " * " + str(e) + ") + (" + str(b) + " * " + str(phi) + ") = " + str(g))
    
    print ("\nraw d = ", d)
    
    print ("\nd = raw d (mod phi) = ", d % phi)
    
    return d % phi
   

In [13]:
d = my_calculate_d(e,phi)


d * e + b * phi = gcd(e,phi) = 1

(288493875763019341 * 5066291434781220101) + (-219363791826143932 * 6662877403775933920) = 1

raw d =  288493875763019341

d = raw d (mod phi) =  288493875763019341


In [14]:
my_print_number("d", d)


 d 

decimal: 288,493,875,763,019,341 

number of digits: 18 

hex: 400efa6c8bafe4d 

number of bits: 59 



In [15]:
m = 1234567890

In [16]:
# encrypt the message m using exponent e to yield ciphertext c

c = power_mod(m, e, n)

In [17]:
my_print_number("c", c)


 c 

decimal: 5,524,095,359,743,005,213 

number of digits: 19 

hex: 4ca9877636dd921d 

number of bits: 63 



In [18]:
# decrypt the ciphertext c using exponent d to yield plaintext m 
# m should match the orginal m set above

m = power_mod(c, d, n)
print (m)

1234567890


Go back and try it with both a positive d and a negative d so you can see how both work.

Go back and try for different bit sizes: 64, 128, 256, 512, 1024, 2048, 4096

Go back and try the short public exponents (3, 7, and 65537) for e